In [ ]:
#Importataan tarvittavat kirjastot
from datetime import datetime
import os
import pandas as pd
import json
from pathlib import Path
from matplotlib import dates as mdates
from matplotlib import pyplot as plt
from __future__ import print_function
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
#Tarkistetaan löytyykö lokaaleille tiedostoille tarvittava /files-kansio
#Jos ei löydy, luodaan se
os.chdir(Path().resolve())
print(os.getcwd())

if(os.path.exists('./files/')):
    pass
else:
    os.mkdir('./files/')

In [ ]:
#Jos löydetään konfiguraatiotiedosto, ladataan sieltä tarvittavat tiedot
load = ''
if(os.path.isfile('./files/conf.json')):
    with open('./files/conf.json', 'r') as f:
        conf_data = json.load(f)
    
    #Kysytään käyttäjältä halutaanko käyttää konfiguraatiotiedostossa olevia tietoja, vai syöttää ne uudelleen
    load = input('Jatka vanhoilla tiedoilla? (y/n): ')

    #Jos halutaan käyttää vanhoja tietoja, ladataan ne muuttujiin
    if(load == 'y'):
        annettu_id = conf_data['given']['id']
        annettu_sheet = conf_data['given']['sheet']
        myyty_id = conf_data['paid']['id']
        myyty_sheet = conf_data['paid']['sheet']
        palautettu_id = conf_data['returned']['id']
        palautettu_sheet = conf_data['returned']['sheet']
        print('Löydetty vanhat konfiguraatiotiedot')
        print('Tiedot:')
        print('annettu_id: ' + annettu_id)
        print('annettu_sheet: ' + annettu_sheet)
        print('myyty_id: ' + myyty_id)
        print('myyty_sheet: ' + myyty_sheet)
        print('palautettu_id: ' + palautettu_id)
        print('palautettu_sheet: ' + palautettu_sheet)
        
#Jos ei löydetä konfiguraatiotiedostoa tai käyttäjä haluaa syöttää tiedot uudelleen, kysytään käyttäjältä tarvittavat tiedot
if(os.path.isfile('./files/conf.json') == False or load == 'n'):
    annettu_id = input('Anna Google Sheet ID jaettujen ja kokouksissa myytyjen kalenterien taulukkoon: ')
    annettu_sheet = input('Anna Taulukon nimi (esim. Taulukko1): ')
    myyty_id = input('Anna Google Sheet ID myytyjen/maksettujen kalenterien taulukkoon: ')
    myyty_sheet = input('Anna Taulukon nimi: ')
    palautettu_id = input('Anna Google Sheet ID palautettujen taulukkoon: ')
    palautettu_sheet = input('Anna Taulukon nimi: ')

    conf_data = {
        'given': {
            'id': annettu_id,
            'sheet': annettu_sheet
        },
        'paid': {
            'id': myyty_id,
            'sheet': myyty_sheet
        },
        'returned': {
            'id': palautettu_id,
            'sheet': palautettu_sheet
    }
    }

    #Tallennetaan konfiguraatiotiedosto syötetyillä tiedoilla
    with open('./files/conf.json', 'w') as f:
        json.dump(conf_data, f)

    print('Tallennettu konfiguraatiotiedot')


In [ ]:
# ladataan tiedot jaetuista, myydyistä ja palautetuista kalenterista sheetsistä

print('Haetaan tiedot sheetsistä...')
given_url = f'https://docs.google.com/spreadsheets/d/{annettu_id}/gviz/tq?tqx=out:csv&sheet={annettu_sheet}'
given_df  = pd.read_csv(given_url, sep=',', encoding='utf-8').apply(lambda x: x.str.strip() if x.dtype == "object" else x)

paid_url  = f'https://docs.google.com/spreadsheets/d/{myyty_id}/gviz/tq?tqx=out:csv&sheet={myyty_sheet}'
paid_df   = pd.read_csv(paid_url, sep=',', encoding='utf-8').apply(lambda x: x.str.strip() if x.dtype == "object" else x)

returned_url = f'https://docs.google.com/spreadsheets/d/{palautettu_id}/gviz/tq?tqx=out:csv&sheet={palautettu_sheet}'
returned_df  = pd.read_csv(returned_url, sep=',', encoding='utf-8').apply(lambda x: x.str.strip() if x.dtype == "object" else x)

print('Tiedot haettu')

# muutetaan päivämäärät oikeaan muotoon
given_df.columns = given_df.columns.str.strip()
given_df['Aikaleima'] = pd.to_datetime(given_df['Aikaleima'], format='%d.%m.%Y klo %H.%M.%S')
given_df = given_df.dropna(subset=['Aikaleima'])
given_df = given_df.sort_values(by=['Aikaleima'])
given_df = given_df.reset_index(drop=True)
given_df

In [ ]:
#TL;DR
#Verrataan ladattuja tiedostoja lokaaleihin, ja ilmoitetaan kuinka paljon ladatuissa tiedostoissa on uutta dataa

#Tarkistetaan löytyyko lokaalia tiedostoa jaetuista kalentereista
#Jos löytyy, palautetaan lokaalin tiedoston ja sheetsistä ladatun tiedoston rivien erotus
#Jos ei löydy, palautetaan sheetsistä ladatun tiedoston rivien määrä
if(os.path.exists('./files/given_raw.csv')):
    old_csv = pd.read_csv('./files/given_raw.csv', sep=',',
                          encoding='utf-8', index_col=0)
    delta = len(given_df) - len(old_csv)
else:
    delta = len(given_df)
if(delta > 0):
    given_df.to_csv('./files/given_raw.csv', sep=',', encoding='utf-8')
print(f'Uusia rivejä annetuissa: {delta}')

#Tarkistetaan löytyyko lokaalia tiedostoa myydyistä (maksetuista) kalentereista
#Jos löytyy, palautetaan lokaalin tiedoston ja sheetsistä ladatun tiedoston rivien erotus
#Jos ei löydy, palautetaan sheetsistä ladatun tiedoston rivien määrä
if(os.path.exists('./files/paid_raw.csv')):
    old_csv = pd.read_csv('./files/paid_raw.csv', sep=',',
                          encoding='utf-8', index_col=0)
    delta = len(paid_df) - len(old_csv)
else:
    delta = len(paid_df)
if(delta > 0):
    paid_df.to_csv('./files/paid_raw.csv', sep=',', encoding='utf-8')
print(f'Uusia rivejä myydyissä: {delta}')

#Tarkistetaan löytyyko lokaalia tiedostoa palautetuista kalentereista
#Jos löytyy, palautetaan lokaalin tiedoston ja sheetsistä ladatun tiedoston rivien erotus
#Jos ei löydy, palautetaan sheetsistä ladatun tiedoston rivien määrä
if(os.path.exists('./files/returned_raw.csv')):
    old_csv = pd.read_csv('./files/returned_raw.csv', sep=',',
                          encoding='utf-8', index_col=0)
    delta = len(returned_df) - len(old_csv)
else:
    delta = len(returned_df)
if(delta > 0):
    returned_df.to_csv('./files/returned_raw.csv', sep=',', encoding='utf-8')
print(f'Uusia rivejä palautetuissa: {delta}')

In [ ]:
#TL;DR
#Parsettaa forms-vastauksista tiedto siitä, kuinka monta kalenteria on jaettu ja myyty kokouksessa minäkin kirjauspäivänä

#Luodaan uusi dataframe tiedoille
info_by_date = pd.DataFrame(columns=['Aika', 'Vartio', 'Annettu', 'Myyty kokouksessa'])

#Käydään läpi annetut kalenterit
for i in range(len(given_df)):
    #Haetaan suoraan kirjauksen aikaleima ja sen tehnyt vartio
    info_by_date.loc[i, 'Aika'] = given_df.loc[i, 'Aikaleima'].strftime('%d.%m.%Y %H:%M')
    info_by_date.loc[i, 'Vartio'] = given_df.loc[i, 'Vartio']

    #Jos kokouksessa myyty kalentereita, lisätään myytyjen määrä. Muuten 0
    if(given_df.loc[i, 'Myittekö kokouksessa kalentereita?'] == 'Kyllä'):
        info_by_date.loc[i, 'Myyty kokouksessa'] = given_df.loc[i, 'Kuinka monta kalenteria myitte kokouksessa?']
    else:
        info_by_date.loc[i, 'Myyty kokouksessa'] = 0

    #Summataan kaikki yhdellä kerralla annetut kalenterit
    sum = 0
    #Käydään läpi kaikki solut, jotka sisältävät annettujen kalenterien määrän
    for j in range(4, 25, 2):
        if(given_df.isnull().iloc[i, j]):
            break     
        sum += given_df.iloc[i, j]
    info_by_date.loc[i, 'Annettu'] = int(sum)

#Muutetaan annetut ja myydyt kokouksessa kokonaisluvuiksi
info_by_date = info_by_date.astype({'Annettu': 'int32', 'Myyty kokouksessa': 'int32'})

#Viedään tiedot csv-tiedostoon
info_by_date.to_csv('./files/info_by_date.csv', sep=',', encoding='utf-8', index=False)
info_by_date




In [ ]:
#TL;DR
#Kokoaa tiedot annetuista, myydyistä ja palautetuista kalentereista yhteen henkilöittäin

#Luodaan dataframe tiedoille
info_by_name = pd.DataFrame(columns=['Vartio', 'Nimi', 'Otettu', 'Myyty', 'Palautettu'])

#Käydään läpi annetut kalenterit
for i in range(len(given_df)):
    index = 3
    while(True):
        if(given_df.isnull().iloc[i, index] or index > 22):
            break
        info_by_name.loc[len(info_by_name)] = [given_df.iloc[i, 1], given_df.iloc[i, index], int(given_df.iloc[i, index+1]), 0, 0]
        index += 2

#Yhdistetään monella kerralla annetut kalenterit yhdeksi
info_by_name = info_by_name.groupby(['Nimi','Vartio']).agg({'Otettu': 'sum', 'Myyty': 'sum', 'Palautettu': 'sum'}).reset_index().sort_values(by=['Vartio','Nimi'])

#Yhdistetään maksetut kalenterit oikeille ihmisille
for i in range(len(paid_df)):
    info_by_name.loc[info_by_name['Nimi'] == paid_df.iloc[i, 1], 'Myyty'] += paid_df.iloc[i, 3]/10

#Varoitus, jos koodi ei löydä kalentereita maksanutta annettujen listalta
if(len(paid_df[~paid_df['Nimi'].isin(info_by_name['Nimi'].tolist())]) > 0):
    print("HUOM! Maksanutta ei lötynyt listoilta:")
    print(paid_df[~paid_df['Nimi'].isin(info_by_name['Nimi'].tolist())])

#Yhdistetään palautetut kalenterit oikeille ihmisille
for i in range(len(returned_df)):
    info_by_name.loc[info_by_name['Nimi'] == returned_df.iloc[i, 1], 'Palautettu'] += returned_df.iloc[i, 3]

#Varoitus, jos koodi ei löydä palauttanutta annettujen listalta
if(len(returned_df[~returned_df['Nimi'].isin(info_by_name['Nimi'].tolist())]) > 0):
    print("HUOM! Palauttanutta ei lötynyt listoilta:")
    print(returned_df[~returned_df['Nimi'].isin(info_by_name['Nimi'].tolist())])

#Viedään tiedot csv-tiedostoon
info_by_name.to_csv('./files/info_by_name.csv', sep=',', encoding='utf-8', index=False)

info_by_name

In [ ]:
#TL;DR
#Kokoaa tiedot myydyistä, palautetuista ja annetuista kalentereista yhteen vartioittain

#Luo dataframen tiedoille
info_by_group = pd.DataFrame(columns=['Vartio', 'Annettu', 'Myyty', 'Palautettu', 'Myyty kokouksissa', 'Mobilepaylla', 'Käteisellä', 'SumUpilla'])

#Käy läpi jokaisen kerran, joina on jaettu/myyty kalentereita, ja lisää tiedot dataframeen
for i in range(len(given_df)):
    info_by_group.loc[i, 'Vartio'] = given_df.loc[i, 'Vartio']
    if(given_df.loc[i, 'Myittekö kokouksessa kalentereita?'] == 'Kyllä'):
        info_by_group.loc[i, 'Myyty kokouksissa'] = int(given_df.loc[i, 'Kuinka monta kalenteria myitte kokouksessa?'])
        info_by_group.loc[i, 'Mobilepaylla'] = int(given_df.loc[i, 'Kuinka monta niistä maksettiin MobilePaylla?'])
        info_by_group.loc[i, 'Käteisellä'] = int(given_df.loc[i, 'Kuinka monta niistä maksettiin käteisellä?'])
        info_by_group.loc[i, 'SumUpilla'] = int(given_df.loc[i, 'Kuinka monta niistä maksettiin SumUpilla (Kortinlukijalla)?'])
    else:
        info_by_group.loc[i, 'Myyty kokouksissa'] = 0

    sum = 0
    for j in range(4, 25, 2):
        if(given_df.isnull().iloc[i, j]):
            break     
        sum += given_df.iloc[i, j]
    info_by_group.loc[i, 'Annettu'] = int(sum)

#Laskee yhteen eri aikoina jaetut ja myydyt kalenterit
info_by_group = info_by_group.groupby('Vartio').agg({'Myyty': 'sum', 'Annettu': 'sum','Palautettu': 'sum', 'Myyty kokouksissa': 'sum', 'Mobilepaylla': 'sum', 'Käteisellä': 'sum', 'SumUpilla': 'sum'}).reset_index()

#Lisätään dataframeen maksetut ja palautetut kalenterit
for i in range(len(info_by_group)):
    info_by_group.loc[i, 'Myyty'] = info_by_name[info_by_name['Vartio'] == info_by_group.loc[i, 'Vartio']]['Myyty'].sum()
    info_by_group.loc[i, 'Palautettu'] = info_by_name[info_by_name['Vartio'] == info_by_group.loc[i, 'Vartio']]['Palautettu'].sum()
info_by_group.astype({'Myyty': 'int32', 'Annettu': 'int32', 'Palautettu': 'int32', 'Myyty kokouksissa': 'int32', 'Mobilepaylla': 'int32', 'Käteisellä': 'int32', 'SumUpilla': 'int32'})

#Lisätään summarivi
sum_row = pd.DataFrame(columns=['Vartio', 'Annettu', 'Palautettu', 'Myyty', 'Myyty kokouksissa', 'Mobilepaylla', 'Käteisellä', 'SumUpilla'])
sum_row.loc[0, 'Vartio'] = 'Yhteensä'
sum_row.loc[0, 'Annettu'] = info_by_group['Annettu'].sum()
sum_row.loc[0, 'Myyty'] = info_by_group['Myyty'].sum()
sum_row.loc[0, 'Myyty kokouksissa'] = info_by_group['Myyty kokouksissa'].sum()
sum_row.loc[0, 'Mobilepaylla'] = info_by_group['Mobilepaylla'].sum()
sum_row.loc[0, 'Käteisellä'] = info_by_group['Käteisellä'].sum()
sum_row.loc[0, 'SumUpilla'] = info_by_group['SumUpilla'].sum()
sum_row.loc[0, 'Palautettu'] = info_by_group['Palautettu'].sum()
info_by_group = pd.concat([info_by_group, sum_row], ignore_index=True)

#Viedään tiedot csv-tiedostoon
info_by_group.to_csv('./files/info_by_group.csv', sep=',', encoding='utf-8', index=False)
info_by_group


In [ ]:
#Visualisoi jaettujen kalenterien määrän vartioittain
plt.bar(info_by_group['Vartio'][:-1], info_by_group['Annettu'][:-1], label='Annettu')
plt.gcf().autofmt_xdate()
plt.gca().bar_label(plt.gca().containers[0], label_type='edge')
plt.legend() 
plt.show()


In [ ]:
#Visualisoi kokouksissa myytyjen kalenterien määrän vartioittain
plt.bar(info_by_group['Vartio'][:-1], info_by_group['Myyty kokouksissa'][:-1], label='Myyty kokouksissa')
plt.gcf().autofmt_xdate()
plt.gca().bar_label(plt.gca().containers[0], label_type='edge')
plt.legend()
plt.show()

In [ ]:
#Visualisoi myytyjen (rahat maksettu) kalenterien määrän vartioittain
plt.bar(info_by_group['Vartio'][:-1], info_by_group['Myyty'][:-1], label='Myyty')
plt.gcf().autofmt_xdate()
plt.gca().bar_label(plt.gca().containers[0], label_type='edge')
plt.legend()
plt.show()

In [ ]:
#Tekee kaavion, joka vertailee annettuja kalentereita eri vuosina.
#Vaatii edellisten vuosien tiedot kansioon files/YYYY_data.csv muodossa
#Vanhojen tietojen sarakkeeet: Aika, Määrä

#Lisää vuosia voi implemetoida koodiin alla olevan mukaisesti

data_22 = info_by_date[['Aika', 'Annettu']].copy()
data_21 = pd.read_csv('./files/2021_data.csv', sep=',', encoding='utf-8')
data_20 = pd.read_csv('./files/2020_data.csv', sep=',', encoding='utf-8')

data_22['Aika'] = pd.to_datetime(data_22['Aika'], format='%d.%m.%Y %H:%M')
data_21['Aika'] = pd.to_datetime(data_21['Aika'], format='%d.%m.%Y')
data_20['Aika'] = pd.to_datetime(data_20['Aika'], format='%d.%m.%Y')

#Siirretään aikoja x vuotta eteenpäin, jotta ne näkyvät samalla aikavälillä
data_21['Aika'] = data_21['Aika'] + pd.DateOffset(years=1)
data_20['Aika'] = data_20['Aika'] + pd.DateOffset(years=2)

data_22 = data_22.resample('D', on='Aika').sum()
data_21 = data_21.resample('D', on='Aika').sum()
data_20 = data_20.resample('D', on='Aika').sum()

data_22['Cumsum'] = data_22['Annettu'].cumsum()
data_21['Cumsum'] = data_21['Määrä'].cumsum()
data_20['Cumsum'] = data_20['Määrä'].cumsum()

plt.plot(data_22.index, data_22['Cumsum'], label='2022')
plt.plot(data_21.index, data_21['Cumsum'], label='2021')
plt.plot(data_20.index, data_20['Cumsum'], label='2020')

plt.legend()
plt.grid()
plt.gcf().autofmt_xdate()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.'))
plt.show()

In [ ]:
#Vajavaisesti toimiva systeemi vertailuun edellisten vuosien kesken

"""
diff_21_abs = (data_22["Cumsum"].iloc[-1] - data_21.loc[dt.datetime.now().strftime("%Y-%m-%d")]['Cumsum'])
diff_21_rel = (data_22["Cumsum"].iloc[-1] - data_21.loc[dt.datetime.now().strftime("%Y-%m-%d")]['Cumsum']) / data_21.loc[dt.datetime.now().strftime("%Y-%m-%d")]['Cumsum'] * 100
diff_20_abs = (data_22["Cumsum"].iloc[-1] - data_20.loc[dt.datetime.now().strftime("%Y-%m-%d")]['Cumsum'])
diff_20_rel = (data_22["Cumsum"].iloc[-1] - data_20.loc[dt.datetime.now().strftime("%Y-%m-%d")]['Cumsum']) / data_20.loc[dt.datetime.now().strftime("%Y-%m-%d")]['Cumsum'] * 100

def sign(x):
    if(x >= 0):
        return 'enemmän'
    else:
        return 'vähemmän'

print(
f'''
Verrattuna tähän aikaan viime vuonna, on jaettu {abs(diff_21_abs):.0f} kalenteria {sign(diff_21_abs)}, eli {diff_21_rel:.0f}% {sign(diff_21_abs)}.
Verrattuna tähän aikaan kaksi vuotta sitten, on jaettu {abs(diff_20_abs):.0f} kalenteria {sign(diff_20_abs)}, eli {diff_20_rel:.0f}% {sign(diff_20_abs)}.
''')
"""

In [ ]:
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

gauth.LoadCredentialsFile("./files/mycreds.txt")
if gauth.credentials is None:
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    gauth.Refresh()
else:
    gauth.Authorize()
gauth.SaveCredentialsFile("./files/mycreds.txt")

if(os.path.isfile('./files/conf.json')):
    with open('./files/conf.json', 'r') as f:
        conf_data = json.load(f)
    
    if("info_by_date" in conf_data["GoogleDrive"]["sheet_ids"]):
        info_by_date_id = conf_data["GoogleDrive"]["sheet_ids"]["info_by_date"]
    else:
        info_by_date_id = input("Anna info_by_date -tiedoston id Drivessä: ")
    
    if("info_by_name" in conf_data["GoogleDrive"]["sheet_ids"]):
        info_by_name_id = conf_data["GoogleDrive"]["sheet_ids"]["info_by_name"]
    else:
        info_by_name_id = input("Anna info_by_name -tiedoston id Drivessä: ")

    if('info_by_group' in conf_data["GoogleDrive"]["sheet_ids"]):
        info_by_group_id = conf_data["GoogleDrive"]["sheet_ids"]["info_by_group"]
    else:
        info_by_group_id = input("Anna info_by_group -tiedoston id Drivessä: ")

    if("parent_folder_id" in conf_data["GoogleDrive"]):
        parent_folder_id = conf_data["GoogleDrive"]["parent_folder_id"]
    else:
        parent_folder_id = input("Anna tiedostojen kansion id Drivessä: ")
    
    conf_data["GoogleDrive"]["sheet_ids"]["info_by_date"] = info_by_date_id
    conf_data["GoogleDrive"]["sheet_ids"]["info_by_name"] = info_by_name_id
    conf_data["GoogleDrive"]["sheet_ids"]["info_by_group"] = info_by_group_id
    conf_data["GoogleDrive"]["parent_folder_id"] = parent_folder_id

    with open('./files/conf.json', 'w') as f:
        json.dump(conf_data, f)      
else:
    info_by_date_id = input("Anna info_by_date -tiedoston id Drivessä: ")
    info_by_name_id = input("Anna info_by_name -tiedoston id Drivessä: ")
    info_by_group_id = input("Anna info_by_group -tiedoston id Drivessä: ")
    parent_folder_id = input("Anna tiedostojen kansion id Drivessä: ")

    conf_data["GoogleDrive"]["sheet_ids"]["info_by_date"] = info_by_date_id
    conf_data["GoogleDrive"]["sheet_ids"]["info_by_name"] = info_by_name_id
    conf_data["GoogleDrive"]["sheet_ids"]["info_by_group"] = info_by_group_id
    conf_data["GoogleDrive"]["parent_folder_id"] = parent_folder_id

    with open('./files/conf.json', 'w') as f:
        json.dump(conf_data, f)




upload_file_list = [
    {'path': './files/info_by_group.csv', 'title': 'info_by_group.csv',
        'id': info_by_group_id},
    {'path': './files/info_by_date.csv', 'title': 'info_by_date.csv',
        'id': info_by_date_id},
    {'path': './files/info_by_name.csv', 'title': 'info_by_name.csv',
        'id': info_by_name_id}
]

for file in upload_file_list:
    file1 = drive.CreateFile({'parents': [{'id': parent_folder_id}], 'title': file['title'], 'id': file['id']})
    file1.SetContentFile(file['path'])
    file1.Upload()
    print('Uploaded file with ID {}'.format(file1.get('id')))